<a href="https://colab.research.google.com/github/xKDR/Shedding-light-on-the-Russia-Ukraine-war/blob/main/reproducible_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WORK IN PROGRESS

In [20]:
using Pkg
Pkg.add(["GeoJSON", "NighttimeLights", "Rasters", "CSV", "Dates", "DataFrames"])

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
Precompiling project...
  17718.1 ms  ? NighttimeLights
  18654.4 ms  ? Rasters → RastersMakieExt


In [58]:
;git clone https://github.com/xKDR/Shedding-light-on-the-Russia-Ukraine-war.git

fatal: destination path 'Shedding-light-on-the-Russia-Ukraine-war' already exists and is not an empty directory.


In [61]:
run(`apt-get install git-lfs -y`)
run(`git lfs install`)
cd("Shedding-light-on-the-Russia-Ukraine-war")
run(`git lfs pull`)

Reading package lists...
Building dependency tree...
Reading state information...
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Git LFS initialized.


Process(`git lfs pull`, ProcessExited(0))

In [63]:
run(`ls -lh /content/Shedding-light-on-the-Russia-Ukraine-war/data/raster/rad_cropped`)

total 2.6G
-rw-r--r-- 1 root root  20M Aug 22 10:54 radiance_2014-01-01T00:00:00.tif
-rwxr-xr-x 1 root root  22M Aug 22 10:54 radiance_2014-02-01T00:00:00.tif
-rwxr-xr-x 1 root root  21M Aug 22 10:54 radiance_2014-03-01T00:00:00.tif
-rwxr-xr-x 1 root root  23M Aug 22 10:53 radiance_2014-04-01T00:00:00.tif
-rwxr-xr-x 1 root root  16M Aug 22 10:54 radiance_2014-05-01T00:00:00.tif
-rwxr-xr-x 1 root root 9.9M Aug 22 10:54 radiance_2014-06-01T00:00:00.tif
-rwxr-xr-x 1 root root  13M Aug 22 10:54 radiance_2014-07-01T00:00:00.tif
-rwxr-xr-x 1 root root  20M Aug 22 10:54 radiance_2014-08-01T00:00:00.tif
-rwxr-xr-x 1 root root  24M Aug 22 10:53 radiance_2014-09-01T00:00:00.tif
-rwxr-xr-x 1 root root  23M Aug 22 10:53 radiance_2014-10-01T00:00:00.tif
-rwxr-xr-x 1 root root  22M Aug 22 10:54 radiance_2014-11-01T00:00:00.tif
-rwxr-xr-x 1 root root  24M Aug 22 10:53 radiance_2014-12-01T00:00:00.tif
-rwxr-xr-x 1 root root  23M Aug 22 10:53 radiance_2015-01-01T00:00:00.tif
-rwxr-xr-x 1 root root  22M

Process(`ls -lh /content/Shedding-light-on-the-Russia-Ukraine-war/data/raster/rad_cropped`, ProcessExited(0))

In [21]:
using GeoJSON
using GeoDataFrames
using DataFrames
using NighttimeLights
using Rasters
using CSV
using Dates

In [42]:
# Define path constants
FIGURES_DIRECTORY = "figures"
TABLES_DIRECTORY = "tables"
VECTOR_DIRECTORY = "data/vector"
TABULAR_DIRECTORY = "data/tabular"
RADIANCE_RASTER_DIRECTORY = "/content/Shedding-light-on-the-Russia-Ukraine-war/data/raster/rad_cropped"
CFOBS_RASTER_DIRECTORY = "/content/Shedding-light-on-the-Russia-Ukraine-war/data/raster/cf_cropped"

"/content/Shedding-light-on-the-Russia-Ukraine-war/data/raster/cf_cropped"

In [24]:
function read_vector(filename)
# Use GeoJSON.read, which expects an IO
    geo = open(filename) do f
        GeoJSON.read(f)
    end

    # Now extract features
    features = geo.features

    # Build DataFrame with properties + geometry
    return DataFrame([merge(Dict(f.properties), Dict(:geometry => f.geometry)) for f in features])
end

read_vector (generic function with 1 method)

In [25]:
russia_state_split = read_vector("/content/Shedding-light-on-the-Russia-Ukraine-war/data/vector/russia_processing.geojson")

Row,geometry,NAME_1,NL_NAME_1,GID_0,VARNAME_1,HASC_1,ISO_1,path,TYPE_1,GID_1,COUNTRY,CC_1,ENGTYPE_1,layer
,Abstract…,String,String,String,String,String,String,String,String,String,String,String,String,String
1,2D Polygon,Chukot1,Чукотский АОк,RUS,Chukotka|Chukotskiy Aok,RU.CK,NA,"Polygon?crs=EPSG:4326&field=GID_1:string(10,0)&field=GID_0:string(10,0)&field=COUNTRY:string(10,0)&field=NAME_1:string(22,0)&field=VARNAME_1:string(34,0)&field=NL_NAME_1:string(61,0)&field=TYPE_1:string(18,0)&field=ENGTYPE_1:string(19,0)&field=CC_1:strin",Avtonomnyy Okrug,RUS.12_1,Russia,7001,Autonomous Province,Single parts
2,2D Polygon,Chukot2_cross,Чукотский АОк,RUS,Chukotka|Chukotskiy Aok,RU.CK,NA,"Polygon?crs=EPSG:4326&field=GID_1:string(10,0)&field=GID_0:string(10,0)&field=COUNTRY:string(10,0)&field=NAME_1:string(22,0)&field=VARNAME_1:string(34,0)&field=NL_NAME_1:string(61,0)&field=TYPE_1:string(18,0)&field=ENGTYPE_1:string(19,0)&field=CC_1:strin",Avtonomnyy Okrug,RUS.12_1,Russia,7001,Autonomous Province,Single parts
3,2D Polygon,Chukot3,Чукотский АОк,RUS,Chukotka|Chukotskiy Aok,RU.CK,NA,"Polygon?crs=EPSG:4326&field=GID_1:string(10,0)&field=GID_0:string(10,0)&field=COUNTRY:string(10,0)&field=NAME_1:string(22,0)&field=VARNAME_1:string(34,0)&field=NL_NAME_1:string(61,0)&field=TYPE_1:string(18,0)&field=ENGTYPE_1:string(19,0)&field=CC_1:strin",Avtonomnyy Okrug,RUS.12_1,Russia,7001,Autonomous Province,Single parts
4,2D Polygon,Chukot4,Чукотский АОк,RUS,Chukotka|Chukotskiy Aok,RU.CK,NA,"Polygon?crs=EPSG:4326&field=GID_1:string(10,0)&field=GID_0:string(10,0)&field=COUNTRY:string(10,0)&field=NAME_1:string(22,0)&field=VARNAME_1:string(34,0)&field=NL_NAME_1:string(61,0)&field=TYPE_1:string(18,0)&field=ENGTYPE_1:string(19,0)&field=CC_1:strin",Avtonomnyy Okrug,RUS.12_1,Russia,7001,Autonomous Province,Single parts
5,2D Polygon,Chukot5_cross,Чукотский АОк,RUS,Chukotka|Chukotskiy Aok,RU.CK,NA,"Polygon?crs=EPSG:4326&field=GID_1:string(10,0)&field=GID_0:string(10,0)&field=COUNTRY:string(10,0)&field=NAME_1:string(22,0)&field=VARNAME_1:string(34,0)&field=NL_NAME_1:string(61,0)&field=TYPE_1:string(18,0)&field=ENGTYPE_1:string(19,0)&field=CC_1:strin",Avtonomnyy Okrug,RUS.12_1,Russia,7001,Autonomous Province,Single parts
6,2D Polygon,Krasnoyarsk1,Красноярский край,RUS,Krasnoyarskiy Kray|Yeniseisk|Yen,RU.KX,NA,/Users/ayushpatnaik/REPOSITORIES/nightlights/DATA/MAPS/RUSSIA_BOUNDARY/split_states.shp,Kray,RUS.35_1,Russia,6209,Territory,split_states
7,2D Polygon,Krasnoyarsk2,Красноярский край,RUS,Krasnoyarskiy Kray|Yeniseisk|Yen,RU.KX,NA,/Users/ayushpatnaik/REPOSITORIES/nightlights/DATA/MAPS/RUSSIA_BOUNDARY/split_states.shp,Kray,RUS.35_1,Russia,6209,Territory,split_states
8,2D MultiPolygon,Krasnoyarsk3,Красноярский край,RUS,Krasnoyarskiy Kray|Yeniseisk|Yen,RU.KX,NA,/Users/ayushpatnaik/REPOSITORIES/nightlights/DATA/MAPS/RUSSIA_BOUNDARY/split_states.shp,Kray,RUS.35_1,Russia,6209,Territory,split_states
9,2D Polygon,Krasnoyarsk4,Красноярский край,RUS,Krasnoyarskiy Kray|Yeniseisk|Yen,RU.KX,NA,/Users/ayushpatnaik/REPOSITORIES/nightlights/DATA/MAPS/RUSSIA_BOUNDARY/split_states.shp,Kray,RUS.35_1,Russia,6209,Territory,split_states


In [26]:
function safe_centre_of_mass(slice::AbstractArray, state_name::String)
    s = sum(skipmissing(slice))
    if ismissing(s) || isnan(s) || s == 0.0f0
        return [NaN, NaN]
    end
    try
        return NighttimeLights.centre_of_mass(slice)
    catch e
        if isa(e, InexactError)
            println("Warning: CoM calculation failed for '$state_name'. Returning [NaN, NaN].")
            return [NaN, NaN]
        else
            rethrow(e)
        end
    end
end

safe_centre_of_mass (generic function with 1 method)

In [32]:
function process_state(state_row::DataFrameRow)
    state_name = state_row.NAME_1
    state_geometry = state_row.geometry

    radiance, cfobs = readnl(state_geometry, START_DATE, END_DATE;
                            rad_path=RADIANCE_RASTER_DIRECTORY,
                            cf_path=CFOBS_RASTER_DIRECTORY)

    radiance = clean_complete(radiance, cfobs; bgthreshold=4)
    cfobs = nothing # Release memory

    radiance = Raster(Float32.(radiance); missingval=missing)
    radiance[radiance .> 100.0f0] .= 0.0f0 # Cap high radiance values
    radiance = mask(radiance; with=state_geometry)

    dates = Date.(dims(radiance, Ti).val)
    num_timesteps = length(dates)

    if num_timesteps == 0
        println("Warning: No valid data for '$state_name' after cleaning.")
        return DataFrame()
    end

    agg_radiance = [sum(skipmissing(view(radiance, Ti(i)))) for i in 1:num_timesteps]
    com = [safe_centre_of_mass(view(radiance, Ti(i)), state_name) for i in 1:num_timesteps]

    return DataFrame(; dates, aggregate_radiance=agg_radiance, com, state_name)
end

process_state (generic function with 1 method)

In [34]:
all_states_df = DataFrame()

const START_DATE = Date(2014, 1)
const END_DATE = Date(2025, 1)


for i in 1:nrow(russia_state_split)
    state_row = russia_state_split[i, :]
    state_name = state_row.NAME_1
    println("--- Processing state $i/$(nrow(russia_state_split)): $state_name ---")

    try
        state_df = process_state(state_row)
        if !isempty(state_df)
            append!(all_states_df, state_df; cols=:union)
        end
    catch e
        println("ERROR: Failed to process state '$state_name'. Skipping. Error: $e")
    end
    GC.gc()
end

println("--- Processing Complete ---")

# Final aggregation
if !isempty(all_states_df)
    println("Aggregating results...")
    grouped = groupby(all_states_df, :dates)
    aggregate_df = DataFrames.combine(grouped, :aggregate_radiance => sum => :AggregateRadiance)
    sort!(aggregate_df, :dates)

    # only write once if you want
    # CSV.write(AGGREGATE_FILE, aggregate_df)
    println("Aggregation complete, DataFrame in memory.")
else
    println("No data processed, skipping aggregation.")
end

println("Script finished.")


--- Processing state 1/101: Chukot1 ---
ERROR: Failed to process state 'Chukot1'. Skipping. Error: MethodError(RasterSeries, (Any[], Ti{Vector{Union{Nothing, DateTime}}}(Union{Nothing, DateTime}[])), 0x00000000000069d9)
--- Processing state 2/101: Chukot2_cross ---
ERROR: Failed to process state 'Chukot2_cross'. Skipping. Error: MethodError(RasterSeries, (Any[], Ti{Vector{Union{Nothing, DateTime}}}(Union{Nothing, DateTime}[])), 0x00000000000069d9)
--- Processing state 3/101: Chukot3 ---
ERROR: Failed to process state 'Chukot3'. Skipping. Error: MethodError(RasterSeries, (Any[], Ti{Vector{Union{Nothing, DateTime}}}(Union{Nothing, DateTime}[])), 0x00000000000069d9)
--- Processing state 4/101: Chukot4 ---
ERROR: Failed to process state 'Chukot4'. Skipping. Error: MethodError(RasterSeries, (Any[], Ti{Vector{Union{Nothing, DateTime}}}(Union{Nothing, DateTime}[])), 0x00000000000069d9)
--- Processing state 5/101: Chukot5_cross ---
ERROR: Failed to process state 'Chukot5_cross'. Skipping. Erro

LoadError: InterruptException:

In [37]:
russia_state_split.geometry[1]

2D Polygonwith 1 sub-geometries

In [50]:
read_region_as_datacubes(
    russia_state_split.geometry[1];
    start_date=START_DATE,
    end_date=END_DATE,
    radiance_path=RADIANCE_RASTER_DIRECTORY,
    cf_path=CFOBS_RASTER_DIRECTORY
)

LoadError: GDALError (CE_Failure, code 10):
	Pointer 'hDS' is NULL in 'GDALGetRasterCount'.



In [47]:
function read_region_as_datacubes(region;
    start_date,
    end_date,
    radiance_path=RADIANCE_RASTER_DIRECTORY,
    cf_path=CFOBS_RASTER_DIRECTORY
)

    # Read and sort file lists
    rad_files = sort(readdir(radiance_path; join=true))
    cf_files  = sort(readdir(cf_path; join=true))

    # Build cropped rasters
    rad_raster_list = [crop(Raster(f; lazy=true), to=region) for f in rad_files]
    cf_raster_list  = [crop(Raster(f; lazy=true), to=region) for f in cf_files]

    # Make raster series with temporal index
    rad_series = RasterSeries(rad_raster_list, Ti(start_date:end_date))
    cf_series  = RasterSeries(cf_raster_list, Ti(start_date:end_date))

    # Convert to datacubes
    rad_datacube = Rasters.combine(rad_series, Ti)
    cf_datacube  = Rasters.combine(cf_series, Ti)

    return rad_datacube, cf_datacube
end


read_region_as_datacubes (generic function with 1 method)

In [65]:
Raster("/content/Shedding-light-on-the-Russia-Ukraine-war/data/raster/rad_cropped/radiance_2025-01-01T00:00:00.tif")

╭─────────────────────────────╮
│ 9563×2028 Raster{Float32,2} │
├─────────────────────────────┴────────────────────────────────────────── dims ┐
  ↓ X Projected{Float64} LinRange{Float64}(19.610418263550002, 178.98125243614294, 9563) ForwardOrdered Regular Intervals{Start},
  → Y Projected{Float64} LinRange{Float64}(74.98541255741124, 41.19374972955001, 2028) ReverseOrdered Regular Intervals{Start}
├──────────────────────────────────────────────────────────────────── metadata ┤
  Metadata{Rasters.GDALsource} of Dict{String, Any} with 4 entries:
  "units"    => ""
  "offset"   => 0.0
  "filepath" => "/content/Shedding-light-on-the-Russia-Ukraine-war/data/raster/…
  "scale"    => 1.0
├────────────────────────────────────────────────────────────────────── raster ┤
  extent: Extent(X = (19.610418263550002, 178.99791953865), Y = (41.19374972955001, 75.00208333335))
  missingval: 3.4028235f38
  crs: GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","70